In [ ]:

import requests
from bs4 import BeautifulSoup
import csv

# Base URL
base_url = "https://books.toscrape.com/catalogue/page-{}.html"

# Create CSV file
with open("books.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Price", "Availability", "Category", "Product_URL"])

    # Loop through all 50 pages
    for page in range(1, 51):
        print(f"Scraping page {page}...")
        url = base_url.format(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        # Find all book containers
        books = soup.find_all("article", class_="product_pod")

        for book in books:
            # Extract details
            title = book.h3.a["title"]
            price = book.find("p", class_="price_color").text.strip()
            availability = book.find("p", class_="instock availability").text.strip()
            product_link = "https://books.toscrape.com/catalogue/" + book.h3.a["href"]

            # Go inside each book page to get the category
            book_page = requests.get(product_link)
            book_soup = BeautifulSoup(book_page.text, "html.parser")
            category = book_soup.find("ul", class_="breadcrumb").find_all("a")[2].text.strip()

            # Save to CSV
            writer.writerow([title, price, availability, category, product_link])

print("Done! Data saved to 'books.csv'")

In [ ]:

import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import textwrap

# === إعداد الروابط الأساسية للموقع ===
base_url = "https://books.toscrape.com/catalogue/page-{}.html"
first_page_url = "https://books.toscrape.com/catalogue/page-1.html"

# === طلب الصفحة الأولى لاكتشاف عدد الصفحات تلقائيًا ===
response = requests.get(first_page_url)
soup = BeautifulSoup(response.text, "html.parser")
page_text = soup.find("li", class_="current").text.strip()  # مثال: "Page 1 of 50"
total_pages = int(page_text.split("of")[-1].strip())
print(f"📄 تم اكتشاف {total_pages} صفحة في الموقع.")

# === إنشاء ملف CSV لحفظ نتائج الكتب ===
with open("books.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    # ✅ ترتيب الأعمدة الجديد
    writer.writerow(["العنوان", "السعر", "التوفر", "رابط_الكتاب"])

    # === تكرار المرور على جميع الصفحات ===
    for page in range(1, total_pages + 1):
        print(f"🔍 جاري استخراج البيانات من الصفحة {page} من {total_pages} ...")
        url = base_url.format(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        # === العثور على جميع الكتب في الصفحة الحالية ===
        books = soup.find_all("article", class_="product_pod")

        # === استخراج البيانات من كل كتاب ===
        for book in books:
            title = book.h3.a["title"]
            price = book.find("p", class_="price_color").text.strip()
            availability = book.find("p", class_="instock availability").text.strip()
            product_link = "https://books.toscrape.com/catalogue/" + book.h3.a["href"]

            # ✅ ترتيب الأعمدة بنفس التسلسل الجديد
            writer.writerow([title, price, availability, product_link])

print("✅ تم استخراج جميع البيانات وحفظها في الملف (books.csv) بنجاح!")

# === قراءة أول 10 صفوف من الملف وعرضها بطريقة منظمة ===
df = pd.read_csv("books.csv").head(10)

# === تقسيم النصوص الطويلة إلى عدة أسطر لتسهيل القراءة ===
wrap_width = 35
for col in df.columns:
    df[col] = df[col].astype(str).apply(lambda x: textwrap.wrap(x, width=wrap_width) or [""])

# === تجهيز عناوين الأعمدة والصفوف للطباعة ===
headers = [[h] for h in df.columns]
rows = [[row[col] for col in df.columns] for _, row in df.iterrows()]

# === حساب عرض كل عمود حسب أطول نص ===
col_widths = []
for col_idx, col in enumerate(df.columns):
    maxw = max(len(line) for cell in ([headers[col_idx]] + [r[col_idx] for r in rows]) for line in cell)
    col_widths.append(maxw)

# === دالة لضبط المسافات داخل الأعمدة ===
def pad(s, width):
    return s + " " * (width - len(s))

# === طباعة العناوين ===
header_line = " | ".join(pad(headers[i][0], col_widths[i]) for i in range(len(headers)))
sep_line = "-+-".join("-" * col_widths[i] for i in range(len(headers)))
print("\n📚 أول 10 كتب تم استخراجها:\n")
print(header_line)
print(sep_line)

# === طباعة الصفوف مع احترام الأسطر المتعددة داخل الخلايا ===
for r in rows:
    max_lines = max(len(cell) for cell in r)
    for i in range(max_lines):
        line_parts = []
        for col_idx, cell in enumerate(r):
            part = cell[i] if i < len(cell) else ""
            line_parts.append(pad(part, col_widths[col_idx]))
        print(" | ".join(line_parts))
    print(sep_line)

📄 تم اكتشاف 50 صفحة في الموقع.
🔍 جاري استخراج البيانات من الصفحة 1 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 2 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 3 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 4 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 5 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 6 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 7 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 8 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 9 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 10 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 11 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 12 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 13 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 14 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 15 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 16 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 17 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 18 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 19 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 20 من 50 ...
🔍 جاري استخراج البيانات من الصفحة 21 م